In [ ]:
## 지금의 예시는 그냥 예시용이며, 실제 완성 코드는 확연히 다를 것임을 말씀드립니다.
## 필요한 라이브러리 임포트

import os
import warnings
warnings.filterwarnings("ignore")
Text(PDF) Loader

from langchain_community.document_loaders import PyMuPDFLoader

# PyMuPDFLoader 을 이용해 PDF 파일 로드(여기는 지오맥스가 될 예정)
# 여기서 PDF가 아닌 Excel, txt 형식도 가능할 것으로 보이나 데이터 전처리가 필요해 보입니다.

loader = PyMuPDFLoader("labor_low.pdf")
pages = loader.load()
TextSplitter

from langchain.text_splitter import RecursiveCharacterTextSplitter

# 문서를 문장으로 분리
## 청크 크기 500, 각 청크의 50자씩 겹치도록 청크를 나눈다
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
)
docs = text_splitter.split_documents(pages)
Text Vector Embedding

from langchain.embeddings import HuggingFaceEmbeddings

# 문장을 임베딩으로 변환하고 벡터 저장소에 저장(단어가 어떻게 나와있는지 유의)

embeddings = HuggingFaceEmbeddings(
    model_name='BAAI/bge-m3',
    #model_kwargs={'device':'cpu'},
    model_kwargs={'device':'cuda'},
    encode_kwargs={'normalize_embeddings':True},
)
VectorStore(Chroma)

# 벡터 저장소 경로 설정
## 현재 경로에 'vectorstore' 경로 생성
vectorstore_path = 'vectorstore'
os.makedirs(vectorstore_path, exist_ok=True)

# 벡터 저장소 생성 및 저장
vectorstore = Chroma.from_documents(docs, embeddings, persist_directory=vectorstore_path)
# 벡터스토어 데이터를 디스크에 저장
vectorstore.persist()
print("Vectorstore created and persisted")
Model

from langchain_community.chat_models import ChatOllama

# Ollama 를 이용해 로컬에서 LLM 실행
## llama3-ko-instruct 모델 다운로드는 Ollama 사용법 참조
model = ChatOllama(model="llama-3-Korean-Bllossom-8B", temperature=0)
# Retriever
retriever = vectorstore.as_retriever(search_kwargs={'k': 3})

# LangChain
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate


# Prompt 템플릿 생성(여기가 지오맥스에서 원하는 질문을 쓰면 원하는 대답이 나오게끔 해주는 코드)
# Question이 사용자 측이 지오맥스 측에 하는 질문이고, prompt가 지오맥스가 내놓는 대답입니다.
# 여기는 대표님이 말씀하신 재난피해 대비서로 채우면 되지 않을까요?

template = '''지오맥스의 재난대비 챗봇입니다. 모든 대답은 한국어(Korean)으로 대답해줘.":
{context}

Question: {question}
'''

prompt = ChatPromptTemplate.from_template(template)

def format_docs(docs):
    return '\n\n'.join([d.page_content for d in docs])

# RAG Chain 연결
rag_chain = (
    {'context': retriever | format_docs, 'question': RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

# Chain 실행
query = "재난 상황 중 지진/홍수/폭설/가뭄 등이 일어나면 어떻게 해야 하는지 알려줘."
answer = rag_chain.invoke(query)

print("Query:", query)
print("Answer:", answer)

# langchain을 사용한 예시이며, query가 질문, answer가 대답입닌다.
# 좋은 대답을 나오게 하려면 주어진 데이터에 RAG를 사용해서 어떻게 잘 조정하면 되지 않을까요?

## 여기서 추가로 지오맥스 쪽에서는 재난 발생이 일어날 때와 대피 정보를 알고 있으니 음성 데이터 같은 것을 추가로 사용할 수 있으면 더 좋을 듯 합니다.

In [ ]:
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

# 대화 상태를 저장하기 위한 클래스
class ConversationState:
    def __init__(self):
        self.history = []

    def add_message(self, role, message):
        self.history.append(f"{role}: {message}")

    def get_context(self):
        return "\n".join(self.history)

# 대화 상태 객체 생성
conversation_state = ConversationState()

# 프롬프트 템플릿 생성
template = '''지오맥스의 재난대비 챗봇입니다. 모든 대답은 한국어(Korean)으로 대답해줘.
{context}

Question: {question}
'''

prompt = ChatPromptTemplate.from_template(template)

def format_docs(docs):
    return '\n\n'.join([d.page_content for d in docs])

# RAG 체인 연결
def run_chain(question):
    # 대화 문맥 업데이트
    context = conversation_state.get_context()

    rag_chain = (
        {'context': context + "\n" + format_docs(docs), 'question': question}
        | prompt
        | model
        | StrOutputParser()
    )

    answer = rag_chain.invoke(question)

    # 대화 상태 업데이트
    conversation_state.add_message("User", question)
    conversation_state.add_message("Bot", answer)

    return answer

# 모델 초기화
model = ChatOllama(model="llama-3-Korean-Bllossom-8B", temperature=0)

# 대화 예시
query = "재난 상황 중 지진/홍수/폭설/가뭄 등이 일어나면 어떻게 해야 하는지 알려줘."
answer = run_chain(query)

print("Query:", query)
print("Answer:", answer)

# 추가 질문
follow_up_query = "그 상황에서 구체적인 조치가 필요한 부분이 있나요?"
follow_up_answer = run_chain(follow_up_query)

print("Follow-up Query:", follow_up_query)
print("Follow-up Answer:", follow_up_answer)

## 추가 발화와 추가 대답이 있을 경우가 있을 경우 그 경우도 만들어 줘야 합니다.